In [207]:
import pandas as pd
#from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import pdist
import numpy as np
# Read the dataframes from the Excel files
file1_path = r'E:\GeorgiaTech\cse6242\CS6242_Fall_2023_JJBASK\Katherine\pta_mta.xlsx'
file2_path = r'E:\GeorgiaTech\cse6242\CS6242_Fall_2023_JJBASK\Katherine\mta.xlsx'

og_pta = pd.read_excel(file1_path)
og_mta = pd.read_excel(file2_path)
og_mta['gender'].fillna(0, inplace=True)
og_mta = og_mta.dropna(subset=['area'])

# Specify the columns you want to convert to integer
columns_to_convert = ['area', 'gender']  # Replace with your actual column names

# Convert selected columns to integer
og_mta[columns_to_convert] = og_mta[columns_to_convert].astype(int)

df_mta = og_mta.drop(columns=['id','gid', 'name'])
df_pta = og_pta.drop(columns=['id','gid', 'name'])

df_pta_summary = df_pta.describe()
revenue_range = []
revenue_range.append(0)
revenue_range.append(df_pta_summary.loc['25%', 'gross_revenue'])
revenue_range.append(df_pta_summary.loc['75%', 'gross_revenue'])
revenue_range.append(df_pta_summary.loc['max', 'gross_revenue'])
print(revenue_range)
df_pta['revenue_success'] = pd.cut(df_pta['gross_revenue'], revenue_range, labels=[1,2,3])

tickets_range = []
tickets_range.append(0)
tickets_range.append(df_pta_summary.loc['25%', 'tickets_sold'])
tickets_range.append(df_pta_summary.loc['75%', 'tickets_sold'])
tickets_range.append(df_pta_summary.loc['max', 'tickets_sold'])
print(tickets_range)
df_pta['tickets_success'] = pd.cut(df_pta['tickets_sold'], revenue_range, labels=[1,2,3])
df_pta['success_score'] = df_pta['tickets_success'].astype("int") + df_pta['revenue_success'].astype("int")
score_range = [0,2,4,6]
df_pta['success_rate'] = pd.cut(df_pta['success_score'], score_range, labels=["low","medium","high"])

df_pta = df_pta.drop(["tickets_success","revenue_success","success_score","gross_revenue", "tickets_sold"],axis=1)

df_pta.head()

[0, 255899142.0, 595859900.0, 2165280638.0]
[0, 4510868.0, 9381862.0, 26178043.0]


,type,area,gender,uc,lc,cc,rc,success_rate
0,1,222,1,11314,712484,560,48776,medium
1,2,222,0,813,132987,15,3138,medium
2,1,222,2,4319,445371,80,8736,medium
3,1,84,1,516,12381,6,291,low
4,2,222,0,3827,836578,203,12572,medium


In [208]:
def calculate_euclidean_distance(input_row, row):    
  distance = pdist([np.array(input_row),np.array(row)])[0]
  return distance

In [209]:
def find_matching_row_and_success_rate(df, input_row):
    # Calculate Euclidean distances between input row and all rows in df
    distances = []
    for index, row in df.iterrows():
        distance = calculate_euclidean_distance(input_row, df.iloc[index][:-1].astype("int64"))
        distances.append((distance, index))        

    # Sort distances in ascending order
    distances.sort()

    # Extract the index of the closest row
    closest_row_index = distances[0][1]

    # Get the success rate of the closest row
    closest_row_success_rate = df.loc[closest_row_index, 'success_rate']

    return closest_row_success_rate

In [212]:
print(find_matching_row_and_success_rate(df_pta,df_mta.iloc[700]))


medium
